# 判决书文本标记

## 背景

从若干份案件判决书的全文中，逐份提取肇事人、事故发生地、伤亡数量等二十余项信息。

## 方案
    
使用大模型进行文本标记，并用function_call功能来获取结构化数据。

### 问题

判决书全文过长，直接丢给大模型会让其标记效果不好，如“总罚款金额”会识别成“赔偿金”、“单项罚款金额”等错误数据。

#### 解决方案1

正文切割成段落或句子，从这些段落和句子中摘要成半结构化数据。

#### 解决方案2

1. 使用大模型读取整篇文本和20个字段的说明，为每个问题字段给出合适文本 
    例如：
    relevant_text:dict[str,list[str]] = {
        "案件类型":[
            "对应文本1",
            "对应文本2"
        ]
        “肇事人”：[
            "对应文本1"
        ]
    }
    提示词为：将原始文本分段，让它从20个字段中选出来最相关的字段标记这段文本
    
2. 将问题字段与对应文本发送给大模型，大模型给出合适结果

注：这个方案是RAG方案的简化版，RAG方案所做的同样是找到20个字段各自对应的文本，但它（找文本这一步的）实现方式是通过向量搜索（通过词嵌入等方式找到最对应文本段）完成的，然后发送给大模型整合答案。RAG之所以这样做是因为被搜索的文本数量很多，大模型上下文长度不够，所以只好用其他方式代替。但在此处，被搜索的文本长度是可以被直接读的，所以用大模型代替RAG部分，找到对应的文本，再次拿着这个文本去询问大模型（文本越短，结果越准）。



In [ ]:
"""
此处代码无法正常运行，后续会有补充，只是作为示例进行演示，最终版本为可运行的核心代码片段
"""
# # 解决方案2的代码

# ## 步骤1： 大模型读取文本和字段说明，为每个问题字段给出合适文本
# tools = [
#         {
#             "type":"function",
#             "function": {
#                 "name": "associate_field_with_text",
#                 "description": "将与文本有关联的字段保存",
#                 "parameters": {
#                     "type": "object",
#                     "properties": {
#                         "field":{
#                             "type": "list[string]",
#                             "description": "与所给文本有关联的若干个字段，例如['文书类型','肇事人']等，只给出字段名称即可",
#                         },
#                         "log":{
#                             "type":"string",
#                             "description":"日志，简要给出你的标记依据，你给出这些字段的原因"
#                         }
#                     },
#                 },
#                 "required":[
#                     "field",
#                     "text",
#                 ]
#             }
#         }
#     ]

# def get_response(self,messages,tools):
#     response = Generation.call(
#         model='qwen-max',
#         messages=messages,
#         tools=tools,
#         result_format='message'
#     )
#     return response

# ## 步骤2： 将问题字段与对应文本发送给大模型，大模型给出合适结果

# question_filed = '案件类型'
# chat ( question_filed, relevant_text [ question_filed ] ,prompt )

